<h3>Qatar2022 Squads</h3>
It reads all squads from Qatar2022 WC then create a new clean dbase to use to enhanced data about each player.<br>
Initial thoughs: Web crawls to get place of birth for each player<br>


In [ ]:
import pandas as pd    
import numpy as np
import scipy.stats as stats
import re
import datetime
from datetime import datetime, date, time, timedelta
from pandas.api.types import is_datetime64_any_dtype as is_datetime
import time

In [ ]:
def read_html(base_site, parser='h'):
    #
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:107.0) Gecko/20100101 Firefox/107.0'}    
    #
    response = requests.get(base_site, headers=headers)
    #
    if not response.status_code == 200:
        return None
    try:
        #
        if (parser == 'l'):
            soup = BeautifulSoup(response.content,'lxml')
            
        if (parser == 'h'):
            soup = BeautifulSoup(response.content, "html.parser")
        return (soup)
    except:
        return None
    #
#./. Eo read_html   


def player_wiki1 (fname, lname, flen, sitein=0, debug=0):
    
    if (sitein == 0):
        player = fname + "_" + lname
        if (flen == 1): player = fname    
        if (debug==1): print (player)
        site= "https://en.wikipedia.org/wiki/" + player
    else:
        site = fname
    #
    pob = "dnk"
    #
    
    print(site)
    soup = read_html(site)
    if (soup is None):
        if (debug==1): print ("ERROR: Web site access failed") 
    elif (re.search("Place of birth", soup.prettify())):
        tables = soup.find_all('tbody')
        #
        for table in tables:
            if (re.search("Place of birth", str(table.contents))):
                trs = table.find_all('tr')
                found = 0
                for entry in trs:
                    #print(entry.get_text())
                    txt = entry.get_text()
                    if (re.search("Place of birth", txt)):
                        if (debug==1): print ("Found it", txt)
                        pob = txt
                        found = 1
                        break
                #./. eo for entry in trs
        #./. eo for looping thru tabels looking for PoB
        if (pob != "dnk"):
            pob = pob.replace("\n"," ")
            pob = pob.replace("Place of birth","")
            pob = re.sub(r"^\s+","", pob)
        #./. eo if pob was found
    return(pob)
#
#./. Eo player_wiki1
#

def qwant_scrap(fname, lname, nation, debug=0):
    site = "https://www.qwant.com/?q=" + fname + "+" + lname + "+" + nation + "+footballer+date+of+birth&t=web"
    print (site)
    #
    plink = None
    tsoup = None
    tsoup = read_html(site)
    #
    divs = tsoup.find_all("div", {"class": "Q8spj"})    
    for div in divs:
        try:
            if (re.search(r"https:\/\/\w+\.wikipedia\.org", str(div['domain']))):        
                a = div.find('a')
                plink = a['href']
                if (debug==1): print("Found:", a['href'])
                break
        except:
            return None
    return(plink)
#
#./. eo def qwant_scrap
#

### Read data

In [ ]:
#
xls0  = pd.ExcelFile("SquadLists-English.xlsx")
df = pd.read_excel(xls0)
#
xls1  = pd.ExcelFile("32con.xlsx")
cty_df = pd.read_excel(xls1)


In [ ]:
cty_df['Cty'].iloc[0]

### Dataframe Cleaning

In [ ]:

df.iloc[1]
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
df.reset_index(drop=True, inplace=True)


In [ ]:
df.info()

### Get data from files
* Read each row
* In each row, get data from each column
* Add data to a new dataframe.

I: In this particular file all data was in different columns, so I could not just drop columns w/ NANs

In [ ]:
def rename_staff (entry):
    num = 0
    if (re.match("Head", entry)):        
        pos="HC"
        num=27
    if (re.match("Assistant", entry)):   pos="AC"     
    if (re.match("Goalkeeping",entry)): pos="GC"
    #
    return(pos, num)

cols = list(df.columns)
cty_idx = 0
col_cnt = 0
ply_cnt = 0
#
fifa_df = pd.DataFrame()
#
for t in list(df.index):  
    # 
    for c in cols:
        #
        row_txt = str(df[c].loc[t])
        if (re.match("Unnamed: 0", c)):
            col_cnt = 1
            if (re.match(r"1\b", row_txt)):
                cty = cty_df['Cty'].iloc[cty_idx]
                cty_idx += 1 
                num = 1
                typerow = 0
                print ("I: First Player", cty, num, cty_idx)
                
            elif (re.match(r"^[a-zA-Z]", row_txt) and not re.match(r"^DOB", row_txt)):
                #
                typerow = 1
                (pos, num) = rename_staff(row_txt)
                if num == 0: num = ply_cnt
                if num != 0: ply_cnt = num
                ply_cnt += 1
                
            elif (re.match(r"^\d", row_txt)): # Assume to be a num > 1
                typerow = 0
                num = df[c].loc[t]
        else:
            if (df[c].loc[t] == df[c].loc[t]):             
                if (col_cnt == 1):
                    if (typerow == 0): pos   = df[c].loc[t]
                    if (typerow == 1): pname = df[c].loc[t]                
                        
                if (col_cnt == 2): # Player name
                    if (typerow == 0): pname = df[c].loc[t]
                    if (typerow == 1): fname = df[c].loc[t]                
                        
                if (col_cnt == 3): # First name
                    if (typerow == 0): fname   = df[c].loc[t]
                    if (typerow == 1): lname = df[c].loc[t]                
                        
                if (col_cnt == 4): # Last name
                    if (typerow == 0): lname = df[c].loc[t]
                    if (typerow == 1): 
                        shirt = pos
                        height = 100
                        caps  = 0
                        goals = 0                        
                        dob = "10/10/1971"
                        club  = df[c].loc[t]     
                        # I: Append series to new df  
                        new_row = pd.Series(data={'Num':num, 'Country':cty, 'Position':pos, 'PName':pname, 'FName':fname,
                                              'LName':lname, 'Shirt':shirt, 'DoB':dob,'Club':club,'Height':height,
                                              'Caps':caps, 'Goals':goals}, name='S')    
                        fifa_df = fifa_df.append(new_row, ignore_index=False)
                        
                if (col_cnt == 5): # Shirt
                    shirt = df[c].loc[t]
                if (col_cnt == 6): # DoB
                    dob = df[c].loc[t]
                if (col_cnt == 7): # Club
                    club = df[c].loc[t]
                if (col_cnt == 8): # Height
                    height = df[c].loc[t]
                if (col_cnt == 9): # Caps
                    caps = df[c].loc[t]
                if (col_cnt == 10): # Goals 
                    goals = df[c].loc[t]
                    #
                    # I: Append series to new df  
                    new_row = pd.Series(data={'Num':num, 'Country':cty, 'Position':pos, 'PName':pname, 'FName':fname,
                                              'LName':lname, 'Shirt':shirt, 'DoB':dob,'Club':club,'Height':height,
                                              'Caps':caps, 'Goals':goals}, name='I')    
                    fifa_df = fifa_df.append(new_row, ignore_index=False)
                #
                col_cnt += 1
                
    


In [ ]:
#
# Reorder the columns to match pdf. Country=Col0
fifa_df = fifa_df[['Country', 'Num', 'Position', 'PName', 'FName', 'LName', 'Shirt', 'DoB', 'Height', 'Club', 'Caps', 'Goals']]
#
# Drop the index I/S given by the Dframe creation
fifa_df.reset_index(drop=True, inplace=True)
#
fifa_df['DoB'] = fifa_df['DoB'].apply(lambda x:datetime.strptime(x,'%d/%m/%Y'))
#
#
def get_age (x):
    from datetime import date
    today = date.today()
    age = today.year - x.year - ((today.month, today.day) < (x.month, x.day))
    return(age)

fifa_df['Age'] = fifa_df['DoB'].apply(lambda x:get_age(x))
#
fifa_df['Num']    = fifa_df['Num'].astype(int)
fifa_df['Height'] = fifa_df['Height'].astype(int)
fifa_df['Caps']   = fifa_df['Caps'].astype(int)
fifa_df['Goals']  = fifa_df['Goals'].astype(int)
#
fifa_df["PoB"] = "dnk"
#
fifa_df.info()

In [ ]:
#
# I: Add Place of Birth. It will attempt to find it in wiki first. If not, then use qwant to get
#    the wiki link. The rest will have to be done manually. Transfermarkit shows city but not country of birth
#    Result: From 983 players, this section found PoB for 98.7% of players ...only 12 to find manually
for t in list(fifa_df.index):  
    # 
    if (re.match(r"HC|AC|GC", fifa_df['Position'].loc[t])):
        pob = str(fifa_df['Club'].loc[t])
    else:
        nation = str(fifa_df['Country'].loc[t])
        nation = nation.lower()
        #
        pname = str(fifa_df['PName'].loc[t])
        pname = pname.lower()
        pname = pname.replace("  "," ")
        pname_arr = pname.split(" ")
        if (len(pname_arr) > 1):
            fname = pname_arr[1]
            lname = pname_arr[0]
        else: # Covers for single-name players like Brasil.
            fname = pname_arr[0]
            lname = ""
        #
        fname = fname.capitalize()
        lname = lname.capitalize()
        #
        print("\n", fname, ":", lname)
        #      
        pob = player_wiki1(fname, lname, len(pname_arr))                        
        #
        # I: If not match found thru wikipedia, try looking at qwant => wiki again
        if (pob == "dnk"):            
            if (prev_loop == 1): time.sleep(1) # 1 sec delay
            prev_loop = 1
            plink = qwant_scrap(fname, lname, nation)
            if (plink is not None):
                pob = player_wiki1(plink, lname, len(pname_arr), 1)             
        else:
            prev_loop = 0
    #./. eo if player or coach
    #
    # I: Add pob to fifa_df
    fifa_df['PoB'].loc[t] = pob
    print(pob)
#pob

In [ ]:
fifa_df['PoB'] = fifa_df['PoB'].replace("\[\d+\]+","", regex=True)

def get_country(x):
    nation = x
    x = re.sub(r",\s+", ",", x)
    x = re.sub(r"^\s+", "", x)
    if (re.search(",", x)):
        #print("Found ,")
        pob_arr = x.split(",")
        nation = pob_arr[-1]        
        #print(type(nation))
    return(nation)

fifa_df['PoB_nation'] = fifa_df['PoB'].apply(lambda x: get_country(x))

fifa_df.tail(20)

In [ ]:
fifa_df.to_excel("Players_Qatar2022.xlsx", sheet_name='Sheet', index=False) 